<img src="images/dask_horizontal.svg" align="right" width="30%">

# Bag: Parallel Lists for semi-structured data

Dask-bag excels in processing data that can be represented as a sequence of arbitrary inputs. We'll refer to this as "messy" data, because it can contain complex nested structures, missing fields, mixtures of data types, etc. The *functional* programming style fits very nicely with standard Python iteration, such as can be found in the `itertools` module.

Messy data is often encountered at the beginning of data processing pipelines when large volumes of raw data are first consumed. The initial set of data might be JSON, CSV, XML, or any other format that does not enforce strict structure and datatypes.
For this reason, the initial data massaging and processing is often done with Python `list`s, `dict`s, and `set`s.

These core data structures are optimized for general-purpose storage and processing.  Adding streaming computation with iterators/generator expressions or libraries like `itertools` or [`toolz`](https://toolz.readthedocs.io/en/latest/) let us process large volumes in a small space.  If we combine this with parallel processing then we can churn through a fair amount of data.

Dask.bag is a high level Dask collection to automate common workloads of this form.  In a nutshell

    dask.bag = map, filter, toolz + parallel execution
    
**Related Documentation**

* [Bag documentation](https://docs.dask.org/en/latest/bag.html)
* [Bag screencast](https://youtu.be/-qIiJ1XtSv0)
* [Bag API](https://docs.dask.org/en/latest/bag-api.html)
* [Bag examples](https://examples.dask.org/bag.html)

## Create data

In [1]:
%run prep.py -d accounts

## Setup

Again, we'll use the distributed scheduler. Schedulers will be explained in depth [later](05_distributed.ipynb).

In [2]:
from dask.distributed import Client

client = Client(n_workers=4)

## Creation

You can create a `Bag` from a Python sequence, from files, from data on S3, etc.
We demonstrate using `.take()` to show elements of the data. (Doing `.take(1)` results in a tuple with one element)

Note that the data are partitioned into blocks, and there are many items per block. In the first example, the two partitions contain five elements each, and in the following two, each file is partitioned into one or more bytes blocks.

In [3]:
# each element is an integer
import dask.bag as db
b = db.from_sequence([1, 2, 3, 4, 5, 6, 7, 8, 9, 10], npartitions=2)
b.take(3)

(1, 2, 3)

In [4]:
# each element is a text file, where each line is a JSON object
# note that the compression is handled automatically
import os
b = db.read_text(os.path.join('data', 'accounts.*.json.gz'))
b.take(1)

('{"id": 0, "name": "Edith", "transactions": [{"transaction-id": 424, "amount": 137}, {"transaction-id": 3146, "amount": 132}, {"transaction-id": 8930, "amount": 127}]}\n',)

In [5]:
# Edit sources.py to configure source locations
import sources
sources.bag_url

's3://dask-data/nyc-taxi/2015/yellow_tripdata_2015-01.csv'

In [6]:
# Requires `s3fs` library
# each partition is a remote CSV text file
b = db.read_text(sources.bag_url,
                 storage_options={'anon': True})
b.take(1)

('VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RateCodeID,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount\n',)

## Manipulation

`Bag` objects hold the standard functional API found in projects like the Python standard library, `toolz`, or `pyspark`, including `map`, `filter`, `groupby`, etc..

Operations on `Bag` objects create new bags.  Call the `.compute()` method to trigger execution, as we saw for `Delayed` objects.  

In [7]:
def is_even(n):
    return n % 2 == 0

b = db.from_sequence([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
c = b.filter(is_even).map(lambda x: x ** 2)
c

dask.bag<lambda, npartitions=10>

In [8]:
# blocking form: wait for completion (which is very fast in this case)
c.compute()

[4, 16, 36, 64, 100]

### Example: Accounts JSON data

We've created a fake dataset of gzipped JSON data in your data directory.  This is like the example used in the `DataFrame` example we will see later, except that it has bundled up all of the entires for each individual `id` into a single record.  This is similar to data that you might collect off of a document store database or a web API.

Each line is a JSON encoded dictionary with the following keys

*  id: Unique identifier of the customer
*  name: Name of the customer
*  transactions: List of `transaction-id`, `amount` pairs, one for each transaction for the customer in that file

In [9]:
filename = os.path.join('data', 'accounts.*.json.gz')
lines = db.read_text(filename)
lines.take(3)

('{"id": 0, "name": "Edith", "transactions": [{"transaction-id": 424, "amount": 137}, {"transaction-id": 3146, "amount": 132}, {"transaction-id": 8930, "amount": 127}]}\n',
 '{"id": 1, "name": "Hannah", "transactions": [{"transaction-id": 71, "amount": -842}, {"transaction-id": 177, "amount": -981}, {"transaction-id": 213, "amount": -959}, {"transaction-id": 400, "amount": -1058}, {"transaction-id": 422, "amount": -1095}, {"transaction-id": 593, "amount": -803}, {"transaction-id": 971, "amount": -1005}, {"transaction-id": 978, "amount": -1040}, {"transaction-id": 1049, "amount": -1005}, {"transaction-id": 1075, "amount": -1068}, {"transaction-id": 1094, "amount": -1230}, {"transaction-id": 1203, "amount": -1058}, {"transaction-id": 1250, "amount": -937}, {"transaction-id": 1480, "amount": -1335}, {"transaction-id": 1541, "amount": -1194}, {"transaction-id": 1715, "amount": -1234}, {"transaction-id": 1837, "amount": -1042}, {"transaction-id": 1857, "amount": -869}, {"transaction-id": 19

Our data comes out of the file as lines of text. Notice that file decompression happened automatically. We can make this data look more reasonable by mapping the `json.loads` function onto our bag.

In [10]:
import json
js = lines.map(json.loads)
# take: inspect first few elements
js.take(3)

({'id': 0,
  'name': 'Edith',
  'transactions': [{'transaction-id': 424, 'amount': 137},
   {'transaction-id': 3146, 'amount': 132},
   {'transaction-id': 8930, 'amount': 127}]},
 {'id': 1,
  'name': 'Hannah',
  'transactions': [{'transaction-id': 71, 'amount': -842},
   {'transaction-id': 177, 'amount': -981},
   {'transaction-id': 213, 'amount': -959},
   {'transaction-id': 400, 'amount': -1058},
   {'transaction-id': 422, 'amount': -1095},
   {'transaction-id': 593, 'amount': -803},
   {'transaction-id': 971, 'amount': -1005},
   {'transaction-id': 978, 'amount': -1040},
   {'transaction-id': 1049, 'amount': -1005},
   {'transaction-id': 1075, 'amount': -1068},
   {'transaction-id': 1094, 'amount': -1230},
   {'transaction-id': 1203, 'amount': -1058},
   {'transaction-id': 1250, 'amount': -937},
   {'transaction-id': 1480, 'amount': -1335},
   {'transaction-id': 1541, 'amount': -1194},
   {'transaction-id': 1715, 'amount': -1234},
   {'transaction-id': 1837, 'amount': -1042},
   {'t

### Basic Queries

Once we parse our JSON data into proper Python objects (`dict`s, `list`s, etc.) we can perform more interesting queries by creating small Python functions to run on our data.

In [11]:
# filter: keep only some elements of the sequence
js.filter(lambda record: record['name'] == 'Alice').take(5)

({'id': 15,
  'name': 'Alice',
  'transactions': [{'transaction-id': 984, 'amount': 420},
   {'transaction-id': 1036, 'amount': 468},
   {'transaction-id': 2970, 'amount': 415},
   {'transaction-id': 3071, 'amount': 484},
   {'transaction-id': 3487, 'amount': 458},
   {'transaction-id': 3922, 'amount': 460},
   {'transaction-id': 4264, 'amount': 474},
   {'transaction-id': 5447, 'amount': 430},
   {'transaction-id': 5641, 'amount': 459},
   {'transaction-id': 5907, 'amount': 406},
   {'transaction-id': 5970, 'amount': 488},
   {'transaction-id': 6071, 'amount': 438},
   {'transaction-id': 6303, 'amount': 436},
   {'transaction-id': 8273, 'amount': 485},
   {'transaction-id': 8684, 'amount': 438},
   {'transaction-id': 8940, 'amount': 449},
   {'transaction-id': 9249, 'amount': 422},
   {'transaction-id': 9320, 'amount': 448},
   {'transaction-id': 9414, 'amount': 463},
   {'transaction-id': 9767, 'amount': 443},
   {'transaction-id': 9859, 'amount': 473},
   {'transaction-id': 9870, 'a

In [12]:
def count_transactions(d):
    return {'name': d['name'], 'count': len(d['transactions'])}

# map: apply a function to each element
(js.filter(lambda record: record['name'] == 'Alice')
   .map(count_transactions)
   .take(5))

({'name': 'Alice', 'count': 23},
 {'name': 'Alice', 'count': 78},
 {'name': 'Alice', 'count': 102},
 {'name': 'Alice', 'count': 23},
 {'name': 'Alice', 'count': 47})

In [13]:
# pluck: select a field, as from a dictionary, element[field]
(js.filter(lambda record: record['name'] == 'Alice')
   .map(count_transactions)
   .pluck('count')
   .take(5))

(23, 78, 102, 23, 47)

In [14]:
# Average number of transactions for all of the Alice entries
(js.filter(lambda record: record['name'] == 'Alice')
   .map(count_transactions)
   .pluck('count')
   .mean()
   .compute())

54.89333333333333

### Use `flatten` to de-nest

In the example below we see the use of `.flatten()` to flatten results.  We compute the average amount for all transactions for all Alices.

In [15]:
js.filter(lambda record: record['name'] == 'Alice').pluck('transactions').take(3)

([{'transaction-id': 984, 'amount': 420},
  {'transaction-id': 1036, 'amount': 468},
  {'transaction-id': 2970, 'amount': 415},
  {'transaction-id': 3071, 'amount': 484},
  {'transaction-id': 3487, 'amount': 458},
  {'transaction-id': 3922, 'amount': 460},
  {'transaction-id': 4264, 'amount': 474},
  {'transaction-id': 5447, 'amount': 430},
  {'transaction-id': 5641, 'amount': 459},
  {'transaction-id': 5907, 'amount': 406},
  {'transaction-id': 5970, 'amount': 488},
  {'transaction-id': 6071, 'amount': 438},
  {'transaction-id': 6303, 'amount': 436},
  {'transaction-id': 8273, 'amount': 485},
  {'transaction-id': 8684, 'amount': 438},
  {'transaction-id': 8940, 'amount': 449},
  {'transaction-id': 9249, 'amount': 422},
  {'transaction-id': 9320, 'amount': 448},
  {'transaction-id': 9414, 'amount': 463},
  {'transaction-id': 9767, 'amount': 443},
  {'transaction-id': 9859, 'amount': 473},
  {'transaction-id': 9870, 'amount': 403},
  {'transaction-id': 9879, 'amount': 463}],
 [{'transac

In [16]:
(js.filter(lambda record: record['name'] == 'Alice')
   .pluck('transactions')
   .flatten()
   .take(3))

({'transaction-id': 984, 'amount': 420},
 {'transaction-id': 1036, 'amount': 468},
 {'transaction-id': 2970, 'amount': 415})

In [17]:
(js.filter(lambda record: record['name'] == 'Alice')
   .pluck('transactions')
   .flatten()
   .pluck('amount')
   .take(3))

(420, 468, 415)

In [18]:
(js.filter(lambda record: record['name'] == 'Alice')
   .pluck('transactions')
   .flatten()
   .pluck('amount')
   .mean()
   .compute())

813.8423002186058

### Groupby and Foldby

Often we want to group data by some function or key.  We can do this either with the `.groupby` method, which is straightforward but forces a full shuffle of the data (expensive) or with the harder-to-use but faster `.foldby` method, which does a streaming combined groupby and reduction.

*  `groupby`:  Shuffles data so that all items with the same key are in the same key-value pair
*  `foldby`:  Walks through the data accumulating a result per key

*Note: the full groupby is particularly bad. In actual workloads you would do well to use `foldby` or switch to `DataFrame`s if possible.*

### `groupby`

Groupby collects items in your collection so that all items with the same value under some function are collected together into a key-value pair.

In [19]:
b = db.from_sequence(['Alice', 'Bob', 'Charlie', 'Dan', 'Edith', 'Frank'])
b.groupby(len).compute()  # names grouped by length

[(7, ['Charlie']), (3, ['Bob', 'Dan']), (5, ['Alice', 'Edith', 'Frank'])]

In [20]:
b = db.from_sequence(list(range(10)))
b.groupby(lambda x: x % 2).compute()

[(0, [0, 2, 4, 6, 8]), (1, [1, 3, 5, 7, 9])]

In [21]:
b.groupby(lambda x: x % 2).starmap(lambda k, v: (k, max(v))).compute()

[(0, 8), (1, 9)]

### `foldby`

Foldby can be quite odd at first.  It is similar to the following functions from other libraries:

*  [`toolz.reduceby`](http://toolz.readthedocs.io/en/latest/streaming-analytics.html#streaming-split-apply-combine)
*  [`pyspark.RDD.combineByKey`](http://abshinn.github.io/python/apache-spark/2014/10/11/using-combinebykey-in-apache-spark/)

When using `foldby` you provide 

1.  A key function on which to group elements
2.  A binary operator such as you would pass to `reduce` that you use to perform reduction per each group
3.  A combine binary operator that can combine the results of two `reduce` calls on different parts of your dataset.

Your reduction must be associative.  It will happen in parallel in each of the partitions of your dataset.  Then all of these intermediate results will be combined by the `combine` binary operator.

In [22]:
is_even = lambda x: x % 2
b.foldby(is_even, binop=max, combine=max).compute()

[(0, 8), (1, 9)]

### Example with account data

We find the number of people with the same name.

In [23]:
%%time
# Warning, this one takes a while...
result = js.groupby(lambda item: item['name']).starmap(lambda k, v: (k, len(v))).compute()
print(sorted(result))

[('Alice', 144), ('Alice', 144), ('Alice', 156), ('Alice', 156), ('Bob', 120), ('Bob', 120), ('Bob', 130), ('Bob', 130), ('Charlie', 72), ('Charlie', 72), ('Charlie', 78), ('Charlie', 78), ('Dan', 48), ('Dan', 48), ('Dan', 52), ('Dan', 52), ('Edith', 101), ('Edith', 101), ('Edith', 104), ('Edith', 109), ('Frank', 84), ('Frank', 84), ('Frank', 91), ('Frank', 91), ('George', 124), ('George', 139), ('George', 262), ('Hannah', 131), ('Hannah', 132), ('Hannah', 143), ('Hannah', 143), ('Ingrid', 95), ('Ingrid', 96), ('Ingrid', 104), ('Ingrid', 104), ('Jerry', 156), ('Jerry', 157), ('Jerry', 173), ('Jerry', 174), ('Kevin', 84), ('Kevin', 84), ('Kevin', 91), ('Kevin', 91), ('Laura', 131), ('Laura', 132), ('Laura', 143), ('Laura', 143), ('Michael', 132), ('Michael', 132), ('Michael', 143), ('Michael', 143), ('Norbert', 146), ('Norbert', 146), ('Norbert', 158), ('Norbert', 159), ('Oliver', 60), ('Oliver', 60), ('Oliver', 65), ('Oliver', 65), ('Patricia', 84), ('Patricia', 84), ('Patricia', 91), 

In [24]:
%%time
# This one is comparatively fast and produces the same result.
from operator import add
def incr(tot, _):
    return tot+1

result = js.foldby(key='name', 
                   binop=incr, 
                   initial=0, 
                   combine=add, 
                   combine_initial=0).compute()
print(sorted(result))

[('Alice', 600), ('Bob', 500), ('Charlie', 300), ('Dan', 200), ('Edith', 415), ('Frank', 350), ('George', 525), ('Hannah', 549), ('Ingrid', 399), ('Jerry', 660), ('Kevin', 350), ('Laura', 549), ('Michael', 550), ('Norbert', 609), ('Oliver', 250), ('Patricia', 350), ('Quinn', 343), ('Ray', 460), ('Sarah', 446), ('Tim', 596), ('Ursula', 549), ('Victor', 550), ('Wendy', 511), ('Xavier', 440), ('Yvonne', 643), ('Zelda', 497)]
CPU times: user 156 ms, sys: 4.24 ms, total: 160 ms
Wall time: 594 ms


### Exercise: compute total amount per name

We want to groupby (or foldby) the `name` key, then add up the all of the amounts for each name.

Steps

1.  Create a small function that, given a dictionary like 

        {'name': 'Alice', 'transactions': [{'amount': 1, 'id': 123}, {'amount': 2, 'id': 456}]}
        
    produces the sum of the amounts, e.g. `3`
    
2.  Slightly change the binary operator of the `foldby` example above so that the binary operator doesn't count the number of entries, but instead accumulates the sum of the amounts.

In [25]:
# Your code here...

## DataFrames

For the same reasons that Pandas is often faster than pure Python, `dask.dataframe` can be faster than `dask.bag`.  We will work more with DataFrames later, but from for the bag point of view, they are frequently the end-point of the "messy" part of data ingestion—once the data can be made into a data-frame, then complex split-apply-combine logic will become much more straight-forward and efficient.

You can transform a bag with a simple tuple or flat dictionary structure into a `dask.dataframe` with the `to_dataframe` method.

In [26]:
df1 = js.to_dataframe()
df1.head()

,id,name,transactions
0,0,Edith,"[{'transaction-id': 424, 'amount': 137}, {'tra..."
1,1,Hannah,"[{'transaction-id': 71, 'amount': -842}, {'tra..."
2,2,Dan,"[{'transaction-id': 785, 'amount': 617}, {'tra..."
3,3,Yvonne,"[{'transaction-id': 495, 'amount': 7569}, {'tr..."
4,4,Tim,"[{'transaction-id': 518, 'amount': 1104}, {'tr..."


This now looks like a well-defined DataFrame, and we can apply Pandas-like computations to it efficiently.

Using a Dask DataFrame, how long does it take to do our prior computation of numbers of people with the same name?  It turns out that `dask.dataframe.groupby()` beats `dask.bag.groupby()` more than an order of magnitude; but it still cannot match `dask.bag.foldby()` for this case.

In [27]:
%time df1.groupby('name').id.count().compute().head()

CPU times: user 206 ms, sys: 17.5 ms, total: 224 ms
Wall time: 1.74 s


name
Alice      600
Bob        500
Charlie    300
Dan        200
Edith      415
Name: id, dtype: int64

### Denormalization

This DataFrame format is less-than-optimal because the `transactions` column is filled with nested data so Pandas has to revert to `object` dtype, which is quite slow in Pandas.  Ideally we want to transform to a dataframe only after we have flattened our data so that each record is a single `int`, `string`, `float`, etc..

In [28]:
def denormalize(record):
    # returns a list for every nested item, each transaction of each person
    return [{'id': record['id'], 
             'name': record['name'], 
             'amount': transaction['amount'], 
             'transaction-id': transaction['transaction-id']}
            for transaction in record['transactions']]

transactions = js.map(denormalize).flatten()
transactions.take(3)

({'id': 0, 'name': 'Edith', 'amount': 137, 'transaction-id': 424},
 {'id': 0, 'name': 'Edith', 'amount': 132, 'transaction-id': 3146},
 {'id': 0, 'name': 'Edith', 'amount': 127, 'transaction-id': 8930})

In [29]:
df = transactions.to_dataframe()
df.head()

,id,name,amount,transaction-id
0,0,Edith,137,424
1,0,Edith,132,3146
2,0,Edith,127,8930
3,1,Hannah,-842,71
4,1,Hannah,-981,177


In [30]:
%%time
# number of transactions per name
# note that the time here includes the data load and ingestion
df.groupby('name')['transaction-id'].count().compute()

CPU times: user 177 ms, sys: 31.6 ms, total: 209 ms
Wall time: 1.47 s


name
Alice       32936
Bob         13983
Charlie      7967
Dan          6106
Edith       15314
Frank       17799
George      16168
Hannah      18179
Ingrid      29763
Jerry       30411
Kevin       26963
Laura       12913
Michael     21144
Norbert     28345
Oliver      18423
Patricia     7491
Quinn       10787
Ray         14172
Sarah       22168
Tim         15009
Ursula      17509
Victor      29073
Wendy       20694
Xavier      19819
Yvonne      16183
Zelda       30681
Name: transaction-id, dtype: int64

## Limitations

Bags provide very general computation (any Python function.)  This generality
comes at cost.  Bags have the following known limitations

1.  Bag operations tend to be slower than array/dataframe computations in the
    same way that Python tends to be slower than NumPy/Pandas
2.  ``Bag.groupby`` is slow.  You should try to use ``Bag.foldby`` if possible.
    Using ``Bag.foldby`` requires more thought. Even better, consider creating
    a normalised dataframe.

## Learn More

* [Bag documentation](https://docs.dask.org/en/latest/bag.html)
* [Bag screencast](https://youtu.be/-qIiJ1XtSv0)
* [Bag API](https://docs.dask.org/en/latest/bag-api.html)
* [Bag examples](https://examples.dask.org/bag.html)

## Shutdown

In [31]:
client.shutdown()